<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/retrievers/pathway_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 路径检索器


[Pathway](https://pathway.com/)是一个开放的数据处理框架。它允许您轻松开发数据转换管道和机器学习应用程序，这些应用程序可以与实时数据源和不断变化的数据一起工作。

本笔记本演示了如何使用`LlamaIndex`实时数据索引管道。您可以使用提供的`PathwayRetriever`从LLM应用程序中查询此管道的结果。然而，在幕后，Pathway在每次数据更改时更新索引，从而为您提供始终最新的答案。

在本笔记本中，我们将使用一个[公共演示文档处理管道](https://pathway.com/solutions/ai-pipelines#try-it-out)，该管道可以：

1. 监视多个云数据源的数据更改。
2. 为数据构建一个向量索引。

要创建您自己的文档处理管道，请查看[托管的提供](https://pathway.com/solutions/ai-pipelines)或按照本笔记本中的步骤[构建您自己的](https://pathway.com/developers/user-guide/llm-xpack/vectorstore_pipeline/)。

我们将使用`llama_index.retrievers.pathway.PathwayRetriever`检索器连接到索引，该检索器实现了`retrieve`接口。

本文档中描述的基本管道允许轻松构建存储在云位置中的文件的简单索引。然而，Pathway提供了构建实时数据管道和应用程序所需的一切，包括类似SQL的操作，如在不同数据源之间进行groupby-reductions和连接，数据的基于时间的分组和窗口处理，以及各种连接器。

有关Pathway数据摄入管道和向量存储的更多详细信息，请访问[向量存储管道](https://pathway.com/developers/showcases/vectorstore_pipeline)。


## 先决条件

要使用`PathwayRetrievier`，您必须安装`llama-index-retrievers-pathway`包。


In [ ]:
!pip install llama-index-retrievers-pathway

## 为llama-index创建检索器


要实例化和配置`PathwayRetriever`，您需要提供文档索引管道的`url`或`host`和`port`。在下面的代码中，我们使用一个公开可用的[演示管道](https://pathway.com/solutions/ai-pipelines#try-it-out)，其REST API可以在`https://demo-document-indexing.pathway.stream`上访问。该演示管道从[Google Drive](https://drive.google.com/drive/u/0/folders/1cULDv2OaViJBmOfG5WB0oWcgayNrGtVs)和[Sharepoint](https://navalgo.sharepoint.com/sites/ConnectorSandbox/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2FConnectorSandbox%2FShared%20Documents%2FIndexerSandbox&p=true&ga=1)摄取文档，并维护用于检索文档的索引。


In [ ]:
from llama_index.retrievers.pathway import PathwayRetriever

retriever = PathwayRetriever(
    url="https://demo-document-indexing.pathway.stream"
)
retriever.retrieve(str_or_query_bundle="what is pathway")

**轮到你了！** [获取你的流程管道](https://pathway.com/solutions/ai-pipelines)或上传[新文档](https://chat-realtime-sharepoint-gdrive.demo.pathway.com/)到演示流程管道，然后重试查询！


## 在查询引擎中的使用


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever,
)

In [ ]:
response = query_engine.query("Tell me about Pathway")
print(str(response))

## 构建自己的数据处理流程

在数据科学和机器学习项目中，数据处理是一个至关重要的步骤。在本教程中，我们将学习如何构建自己的数据处理流程，以便有效地准备数据用于建模和分析。

我们将使用Python中的一些流行的库来构建我们的数据处理流程，包括Pandas和Scikit-learn。我们将按照以下步骤进行：

1. 数据收集
2. 数据清洗
3. 特征工程
4. 数据转换
5. 数据建模

让我们开始吧！


### 先决条件

安装 `pathway` 包。然后下载示例数据。


In [ ]:
%pip install pathway
%pip install llama-index-embeddings-openai

In [ ]:
!mkdir -p 'data/'
!wget 'https://gist.githubusercontent.com/janchorowski/dd22a293f3d99d1b726eedc7d46d2fc0/raw/pathway_readme.md' -O 'data/pathway_readme.md'

### 定义Pathway跟踪的数据源


Pathway可以同时监听许多来源，比如本地文件、S3文件夹、云存储和任何数据流，以便进行数据更改的监控。

更多信息请参见[pathway-io](https://pathway.com/developers/api-docs/pathway-io)。


In [ ]:
import pathway as pwdata_sources = []data_sources.append(    pw.io.fs.read(        "./data",        format="binary",        mode="streaming",        with_metadata=True,    )  # 这将创建一个`pathway`连接器，用于跟踪./data目录中的所有文件)# 这将创建一个用于跟踪Google Drive中文件的连接器。# 请按照https://pathway.com/developers/tutorials/connectors/gdrive-connector/上的说明获取凭据# data_sources.append(#     pw.io.gdrive.read(object_id="17H4YpBOAKQzEJ93xmC2z170l0bP2npMy", service_user_credentials_file="credentials.json", with_metadata=True))

### 创建文档索引流水线


让我们创建文档索引流水线。`transformations` 应该是一个以 `Embedding` 转换结束的 `TransformComponent` 列表。

在这个例子中，让我们首先使用 `TokenTextSplitter` 分割文本，然后使用 `OpenAIEmbedding` 进行嵌入。


In [ ]:
from pathway.xpacks.llm.vector_store import VectorStoreServerfrom llama_index.embeddings.openai import OpenAIEmbeddingfrom llama_index.core.node_parser import TokenTextSplitter# 创建OpenAIEmbedding对象并设置embed_batch_size为10embed_model = OpenAIEmbedding(embed_batch_size=10)# 定义transformations_example列表，包括TokenTextSplitter对象和embed_model对象transformations_example = [    TokenTextSplitter(        chunk_size=150,        chunk_overlap=10,        separator=" ",    ),    embed_model,]# 通过VectorStoreServer.from_llamaindex_components创建processing_pipeline对象processing_pipeline = VectorStoreServer.from_llamaindex_components(    *data_sources,    transformations=transformations_example,)# 定义Pathway的主机和端口PATHWAY_HOST = "127.0.0.1"PATHWAY_PORT = 8754# `threaded`以分离模式运行pathway，当从终端或容器中运行时，必须将其设置为False# 有关`with_cache`的更多信息，请查看https://pathway.com/developers/api-docs/persistence-apiprocessing_pipeline.run_server(    host=PATHWAY_HOST, port=PATHWAY_PORT, with_cache=False, threaded=True)

### 将检索器连接到自定义管道


In [ ]:
from llama_index.retrievers.pathway import PathwayRetriever

retriever = PathwayRetriever(host=PATHWAY_HOST, port=PATHWAY_PORT)
retriever.retrieve(str_or_query_bundle="what is pathway")